In [5]:
# Import the required libraries
import os
from keras.preprocessing import image

folders = os.listdir('Train')
# Uncommment the below statement to view the folders
# print(folders)

image_data = []
labels = []
count = 0

for ix in folders:
    path = os.path.join("Train", ix)
    # Uncommment the below statement to view the path to the images and their labels
    # print(path, count) 
    for im in os.listdir(path):
        try:
            img = image.load_img(os.path.join(path,im), target_size = (224,224))
            img_array = image.img_to_array(img)
            image_data.append(img_array)
            labels.append(count)
        except:
            pass
    count += 1


In [6]:
import random

combined_dataset = list(zip(image_data, labels))
random.shuffle(combined_dataset)
image_data[:], labels[:] = zip(*combined_dataset)

In [7]:
from keras.utils import np_utils
import numpy as np
X_train = np.array(image_data)
Y_train = np.array(labels)

Y_train = np_utils.to_categorical(Y_train)

In [8]:
#importing the model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
import numpy as np
print("Imported Successfully!")

Imported Successfully!


In [9]:
model = ResNet50(include_top = False, weights = 'imagenet', input_shape = (224,224,3))
print(model.summary())

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [10]:
#building classifier
av1 = GlobalAveragePooling2D()(model.output)

fc1 = Dense(256, activation = 'relu')(av1)

d1 = Dropout(0.5)(fc1)

fc2 = Dense(10, activation = 'softmax')(d1)

In [11]:
model_new = Model(inputs = model.input, outputs = fc2)
model_new.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [18]:
from tensorflow.keras.applications.resnet50 import preprocess_input

image_path = 'pikachu.jpg'
img = image.load_img(image_path,target_size = (224,224))
x = image.img_to_array(img)
x = np.expand_dims(x,axis=0)
x = preprocess_input(x)

pred = model_new.predict(x)
print(np.argmax(pred))

2


In [20]:
adam = Adam(learning_rate = 0.00003)
model_new.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])

In [21]:
for ix in range(len(model_new.layers)):
    print(ix, model_new.layers[ix])

0 <tensorflow.python.keras.engine.input_layer.InputLayer object at 0x15d464f70>
1 <tensorflow.python.keras.layers.convolutional.ZeroPadding2D object at 0x15d577b20>
2 <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x15d577eb0>
3 <tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x15d5e9130>
4 <tensorflow.python.keras.layers.core.Activation object at 0x15d577f10>
5 <tensorflow.python.keras.layers.convolutional.ZeroPadding2D object at 0x15d621a60>
6 <tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x15d6731c0>
7 <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x15d676b50>
8 <tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x15d680c10>
9 <tensorflow.python.keras.layers.core.Activation object at 0x15d673fa0>
10 <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x15d68a6a0>
11 <tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x15d690ac0>
12 <tensorf

In [22]:
for ix in range(169):
    model_new.layers[ix].trainable = False
print(model_new.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [23]:
hist = model_new.fit(X_train, Y_train, shuffle = True, batch_size = 16, epochs = 8, validation_split = 0.20)

Epoch 1/8
83/83 [==============================] - 550s 7s/step - loss: 2.2235 - accuracy: 0.2876 - val_loss: 0.7148 - val_accuracy: 0.7872
Epoch 2/8
83/83 [==============================] - 562s 7s/step - loss: 0.7135 - accuracy: 0.7772 - val_loss: 0.3525 - val_accuracy: 0.8875
Epoch 3/8
83/83 [==============================] - 551s 7s/step - loss: 0.3030 - accuracy: 0.9194 - val_loss: 0.2396 - val_accuracy: 0.9331
Epoch 4/8
83/83 [==============================] - 548s 7s/step - loss: 0.1602 - accuracy: 0.9710 - val_loss: 0.1698 - val_accuracy: 0.9544
Epoch 5/8
83/83 [==============================] - 511s 6s/step - loss: 0.1112 - accuracy: 0.9760 - val_loss: 0.1582 - val_accuracy: 0.9574
Epoch 6/8
83/83 [==============================] - 403s 5s/step - loss: 0.0781 - accuracy: 0.9805 - val_loss: 0.1447 - val_accuracy: 0.9635
Epoch 7/8
83/83 [==============================] - 369s 4s/step - loss: 0.0547 - accuracy: 0.9860 - val_loss: 0.1314 - val_accuracy: 0.9696
Epoch 8/8
83/83 [===

In [30]:
image_path = 'sparrow.jpeg'
img = image.load_img(image_path,target_size = (224,224))
x = image.img_to_array(img)
x = np.expand_dims(x,axis=0)
x = preprocess_input(x)

pred = model_new.predict(x)
print(np.argmax(pred))

5


['.DS_Store',
 'Spearow',
 'Squirtle',
 'Pikachu',
 'Charmander',
 'Fearow',
 'Aerodactyl',
 'Bulbasaur',
 'Meowth',
 'Psyduck',
 'Dratini']

In [ ]:
folders